Done:
converting files to .s,.o files
extracting a function from .o files -> .txt files
extracting keywords from .txt files, map numbers to symbolic name
assigning values to keywords

Flow of notebook:
1. import stuff, get folderpath
2. copyallsharedfiles (only when required)
3. compilespecificfunc (_bad or _good in functype)
4. printfolder (to create keywords file)
5. filetodictionary (turn the keywords file into dictionaries)
6. vectorizefolder

*folderpath is hardcoded
*Tested with c files
*Not working for c++ files

In [1]:
import os
import subprocess
import shlex
import re
import string
import json

folderpath = '/home/(username)/Desktop/goodcexamples/testcases/000'

print("Cell printed")

This function is used if there are shared files that are not in the same directory as the C/C++ file.
(This function is best used for SARD folders.)
*source is hardcoded

In [2]:
def CopyAllSharedFiles():
    source = "/home/(username)/Desktop/goodcexamples/testcases/shared/104/."
    for path, dirs, files in os.walk(folderpath):
        processArg = "cp -a "+source+" "+path
        args = shlex.split(processArg)
        try:
            checkOut = subprocess.check_output(args,cwd=path)
        except subprocess.CalledProcessError:
            print("Completed")

In [3]:
CopyAllSharedFiles()

Using terminal commands to create .o files.
Using objdump to get the assembly code of a specific function, and write to a text file.
(Assembly language: Intel x86_64)
(The specific function is "\_bad" for now, and thus this will fail on files that have only sink/source/helper functions)
(Use RE?)

In [6]:
def CompileSpecificFunc():
    sharedfiles = []
    for path, dirs, files in os.walk("/home/(username)/Workfiles"):
        for file in files:
            #functype = file[:-2]+"_bad"
            functype = "main"
            if((file.endswith(".c"))and (not(file in sharedfiles))):
                processArg = "gcc -ffunction-sections -c "+file
                args = shlex.split(processArg)
                try:
                    checkOut = subprocess.check_output(args,cwd=path)
                except subprocess.CalledProcessError:
                    print("Unable to create .o file for "+file)
                processArg = "objdump -M intel -w -d --section=.text."+functype+" "+file[:-2]+".o"
                args = shlex.split(processArg)
                try:
                    out = subprocess.check_output(args,cwd=path)
                    filename = functype+".txt"
                    filename = path+"/"+filename
                    outfile = open(filename,'w')
                    outfile.write(out.decode("utf-8"))
                    outfile.close()
                except subprocess.CalledProcessError:
                    print("error")
                    continue

In [7]:
CompileSpecificFunc()

Using terminal commands to create .s files.

In [12]:
def CompileAllFuncs():
    sharedfiles = []
    for path,dirs,files in os.walk("/home/(username)/Desktop/cexamples/testcases/shared/108"):
        for file in files:
            sharedfiles.append(file)      
    for path, dirs, files in os.walk(folderpath):
        for file in files:
            print(file)
            newfilename = file[:-2]+'.s'
            if((file.endswith(".c"))and (not(file in sharedfiles))):
                processArg = "gcc -S -masm=intel "+file
                #processArg = "gcc "+file+" -o "+file[:-2]+" -Wa,-adhln="+newfilename+" -g -fverbose-asm -masm=intel"
                args = shlex.split(processArg)
                #out = subprocess.call(args,cwd=path)
                try:
                    checkOut = subprocess.check_output(args,cwd=path)
                except subprocess.CalledProcessError:
                    print("Completed")

In [ ]:
CompileAllFuncs()

In [8]:
def printfile(filetoprint):
    keywordsfile = "keywords.txt"
    kfread = open(keywordsfile,"r+")
    keywords = kfread.read().splitlines()
    keywords = list(filter(None, keywords))
    kwlistsize = len(keywords)
    #Reads the contents of the filetoprint, and store as an array
    content = []
    with open(filetoprint) as f:
        content = f.readlines()
        content = [x.strip() for x in content]
    #Check that for each line in content, opcodes that are not already in the opcodes array will be appended,
    #and other words that are not registers or hexadecimal numbers will be added to instructions array.
    for line in content:
        if not ("\t" in line):
            continue
        templist = line.split("\t")
        tempstr = templist[2]
        tempstr = " ".join(tempstr.split())
        templist = re.split(' |,',tempstr)
        tempstr = templist[0]
        if not(tempstr in keywords):
            keywords.append(tempstr)
        for i in range(1,len(templist)):
            if not (templist[i] in keywords):
                if not ((all(c in string.hexdigits for c in templist[i]))or(templist[i].startswith("<"))or(templist[i].startswith("0x"))):
                    for j in keywords:
                        if (j in templist[i]):
                            break
                    else:
                        keywords.append(templist[i])
    #Write opcodes array to opcodes file, write instructions array to instructions file
    #Closing 2 files after writing
    inputstring = ""
    for i in range(kwlistsize,len(keywords)):
        inputstring = inputstring + keywords[i] + "\n"
    kfread.write(inputstring)
    kfread.close()

In [9]:
def printFolder():      
    for path, dirs, files in os.walk(folderpath):
        for file in files:
            if(file.endswith("main.txt")):
                filetoprint = path+"/"+file
                printfile(filetoprint)

In [10]:
printFolder()

# Assigning values to words in text files -> dictionaries

In [13]:
def filetodictionary():
    kwdict = {}
    keywordsfile = "keywords.txt"
    kfread = open(keywordsfile,"r")
    keywords = kfread.read().splitlines()
    keywords = list(filter(None,keywords))
    kwlistsize = len(keywords)
    kfread.close()
    for i in range(0,kwlistsize):
        kwdict[keywords[i]] = i+1
    keywordsfile = "keywordsdict.txt"
    kfdread = open(keywordsfile,"w")
    kfdread.write(json.dumps(kwdict))
    kfdread.close()
    
filetodictionary()

In [11]:
#To append the vectors into vectors.txt

def symbolizeAndVectorizeToFile(filevectors):
    #randomfile = "/home/(username)/Workfiles/goodvectors.txt"
    randomfile = "/home/(username)/Workfiles/testvector.txt"
    rf = open(randomfile,"a+")
    #rf.write(json.dumps(numdict))
    for i in range(0,len(filevectors)-1):
        rf.write("%s," % str(filevectors[i]))
    rf.write(str(filevectors[len(filevectors)-1]))
    rf.write("\n")
    #data = json.load(rf)
    rf.close()

In [12]:
#To map numbers to symbolic names e.g num1

def trp(l, n):
    return l[:n] + [0]*(n-len(l))


def symbolizeAndVectorize(filetoopen):
    keywordsfile = "keywordsdict.txt"
    kfdread = open(keywordsfile,"r+")
    keywords = json.load(kfdread)
    
    content = []
    opcodes = []
    filevectors = []
    with open(filetoopen) as f:
        content = f.readlines()
        content = [x.strip() for x in content]
        #content - an array of lines from the file that was opened
    for line in content:
        if not ("\t" in line):
            continue
        templist = line.split("\t")
        tempstr = templist[2]
        tempstr = " ".join(tempstr.split())
        templist = re.split(',',tempstr)
        for i in range(0,len(templist)):
            try:
                hexnum = int(templist[i],16)
                templist[i] = "num"
            except ValueError:
                continue
        templist2 = templist[0].split()
        opcodestr = templist2.pop(0)
        opcodes.append(opcodestr)
        
        tempstr2 = " ".join(templist2)
        templist[0] = tempstr2
        templist.insert(0,opcodestr)
        operand1 = templist[1].split()
        
        operand2 = []
        try:
            operand2 = templist[2].split()
        except IndexError:
            operand2.append('')
        lines = opcodes+operand1+operand2
        opcodes = []
        
        for i in range(0,len(lines)):
            if str(lines[i]).startswith("<"):
                lines[i] = "num"
            elif str(lines[i]).startswith("["):
                lines[i] = "num"
            try:
                hexnum = int(lines[i],16)
                lines[i] = "num"
            except ValueError:
                continue
        
        for j in range(len(lines)):
            if lines[j] in keywords:
                lines[j] = keywords[lines[j]]
            if lines[j] != '':
                filevectors.append(lines[j])
        
        
    kfdread.close()
    symbolizeAndVectorizeToFile(filevectors)

In [13]:
def vectorizeFolder():      
    #for path, dirs, files in os.walk(folderpath):
    for path, dirs, files in os.walk("/home/(username)/Workfiles"):
        for file in files:
            if(file.endswith("main.txt")):
                filetovectorize = path+"/"+file
                symbolizeAndVectorize(filetovectorize)
                
vectorizeFolder()

# The codes below are used for miscellaneous testing.

These codes should not be executed when formally processing data.

In [17]:
#Function because the number of files went out of hand

def showRelevantFilesInFolder():
    count = 0
    for path, dirs, files in os.walk(folderpath):
        for file in files:
            #if(file.endswith("good1.txt")):
            if(file.endswith("main.txt")):
                #print(path+"/"+file)
                count += 1
    print(count)
                
showRelevantFilesInFolder()

27


In [7]:
#A test function for regular expressions

import re
def findFunction(filename):
    with open(filename) as openedfile:
        text = openedfile.read()
        lst = re.findall("^(CWE.*_)?good$",text)
        print(lst)
                    
findFunction("/home/(username)/Desktop/cexamples/testcases/000/063/114/CWE121_Stack_Based_Buffer_Overflow__CWE193_char_alloca_memcpy_03.c")

[]


In [9]:
#Temporary codes used for testing etc
import string
def testFunction():
    s = 'e9'
    #print(all(c in string.hexdigits for c in s))
    print(int(s,16))
    #openfile = open(filename,'rb') 
    #data = openfile.read(noOfBytes)
    #return data
    
testFunction()

233


In [ ]:
#Reversed process

def getAssemFromVect():
    keywordsfile = "keywordsdict.txt"
    kfdread = open(keywordsfile,"r+")
    keywords = json.load(kfdread)
    kfdread.close()
    wordslist = []
    randomfile = "/home/(username)/Workfiles/goodvectors.txt"
    with open(randomfile) as file:
        lines = [line.strip() for line in file]
    for line in lines:
        templist = [x.strip() for x in line.split(',')]
        for item in templist:
            for keyword, value in keywords.items():
                #print("item: "+item)
                #print("value: "+value)
                if value == int(item):
                    wordslist.append(keyword)
        print(wordslist)
        wordslist=[]
    
getAssemFromVect()